<a href="https://colab.research.google.com/github/ExCaLBBR/ExCaLBBR_Projects/blob/main/GeometryBasedDecisions/code/LRrandomizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Find Left-Right randomization for PRaM stimuli presentation

In [ ]:
#@title Install dependancies
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from itertools import combinations
import csv

In [ ]:
def item_randperm(ntimes, nperm):
    '''
      p = item_randperm(nitems,nperm)
      Parameters: number of items, number of random permutations
      Output: a matrix with nperm rows;
      Each row is an index of permuted item positions.

      returns a matrix (n,nitems)
      each row is a random permutation of nitems (labelled 1:nitems)
      produces n such permutations
      the random seed is changed at every call
    '''

    p = np.zeros((nperm, ntimes))
    for i in range(nperm):
        p[i,:] = np.random.permutation(ntimes)

    return p

In [ ]:
#@title Load data, Clean dataframes, and mask out participants who didn't complete all parts of the study session
#Data path
path = 'https://github.com/ExCaLBBR/ExCaLBBR_Projects/raw/main/SocioenvironmentalGeometry/'

# Load Demographic PRaM data
datDemo = pd.read_csv (path + 'data/demographic.csv', header=1)
datPRaM = pd.read_csv (path + 'data/df_taskPRaM.csv', header=0)
pairLab = pd.read_csv (path + 'data/PRaM_pairLabels.csv', header=None)
news = pd.read_csv (path + 'data/df_News_coded.csv', header=1)

#remove misc row and incomplete sessions
datDemo = datDemo.drop(labels=0, axis=0)
datDemo = datDemo.reset_index(drop=True)
datDemo['Question Key'] = [sub[: -2] for sub in datDemo['Question Key']]
datPRaM = datPRaM.drop(['Participant Private ID'], axis=1)
datPRaM = datPRaM.reset_index(drop=True)
news = news.drop(labels=0, axis=0)
news = news.reset_index(drop=True)

#Identify participants who did not complete all measures
lst1 = datDemo['Question Key']
lst2 = news['Question Key']
exPart = list(set(lst1) - set(lst2))
mask = datDemo['Question Key'].str.contains(exPart[0], na=True) #If the number of participants to exclude exceeds 1 you will need a for loop
datDemo = datDemo[~mask]
datPRaM = datPRaM[~mask]

#reset index just in case
datDemo = datDemo.reset_index(drop=True)
datPRaM = datPRaM.reset_index(drop=True)
news = news.reset_index(drop=True)

#Sort pairs accoring to prefered combination
words = ['police', 'firefighter', 'neighbors(yours)', 'conservatives(political)', 'liberals(political)', 'healthcare', 'voting', 'immigration', 'religion', 'science', 'anger', 'fear', 'joy', 'love', 'sadness', 'trust']
combinations_list = [list(c) for c in combinations(words, 2)]

# re-organize the dataframe according to the word combination
PairLabel = []
x = []                  #index orders are saved as list
for i in range(len(combinations_list)):

    # Extract pair rating
    idx0 = pairLab[pairLab.iloc[:,1].str.contains(combinations_list[i][0].replace('(', r'\(').replace(')', r'\)'))].index
    idx1 = pairLab[pairLab.iloc[:,2].str.contains(combinations_list[i][1].replace('(', r'\(').replace(')', r'\)'))].index
    pIndx = idx0.intersection(idx1)

    # Extract pair label
    PairLabeli = combinations_list[i][0] + '-' + combinations_list[i][1]
    PairLabel.append(PairLabeli)
    x.append(pIndx[0])

# Pair label rating
datPRaM_conSort = datPRaM.iloc[:,x]
datPRaM_conSort = datPRaM_conSort.rename(columns=dict(zip(datPRaM_conSort.columns, PairLabel)))
PairLabel = pd.DataFrame(PairLabel)

In [ ]:
#@title Compute left/right simulus presentation statistics
fullLeftList = [c for (c,z) in combinations_list]

wLeftPerc = []
for w in words:
  leftTally = [t for t in fullLeftList if t == w]
  wLeftPerc.append((len(leftTally)/(len(words)-1))*100)
print('mean probability of left position = ', np.mean(wLeftPerc))
print('The probabilities range from', min(wLeftPerc), 'to', max(wLeftPerc))

mean probability of left position =  50.0
The probabilities range from 0.0 to 100.0


In [ ]:
diffMax = 100
while diffMax > 20:
  LRrandomizer = item_randperm(2, 120)
  reRandomizedList = [[combinations_list[i][0],combinations_list[i][1]] if value[0]==0 else [combinations_list[i][1],combinations_list[i][0]] for i,value in enumerate(LRrandomizer)]

  fullLeftList = [c for (c,z) in reRandomizedList]

  wLeftPerc = []
  for w in words:
    leftTally = [t for t in fullLeftList if t == w]
    wLeftPerc.append((len(leftTally)/(len(words)-1))*100)
  diffMax = max(wLeftPerc)-min(wLeftPerc)
  print('mean probability of left position = ', np.mean(wLeftPerc))
  print('The probabilities range from', min(wLeftPerc), 'to', max(wLeftPerc))
  print('The range value is:', diffMax)

mean probability of left position =  50.0
The probabilities range from 26.666666666666668 to 80.0
The range value is: 53.33333333333333
mean probability of left position =  50.0
The probabilities range from 20.0 to 66.66666666666666
The range value is: 46.66666666666666
mean probability of left position =  50.0
The probabilities range from 33.33333333333333 to 73.33333333333333
The range value is: 40.0
mean probability of left position =  49.99999999999999
The probabilities range from 33.33333333333333 to 73.33333333333333
The range value is: 40.0
mean probability of left position =  50.0
The probabilities range from 33.33333333333333 to 80.0
The range value is: 46.66666666666667
mean probability of left position =  50.0
The probabilities range from 33.33333333333333 to 73.33333333333333
The range value is: 40.0
mean probability of left position =  50.0
The probabilities range from 33.33333333333333 to 80.0
The range value is: 46.66666666666667
mean probability of left position =  50.0